In [2]:
#Para crear y análizar redes
import networkx as nx
#Para leer desde un archivo
import pandas as pd 
#Para generar errores aleatorios
import random
# using datetime module
import datetime

In [3]:
def imprimir_hora():
    # ct stores current time
    ct = datetime.datetime.now()
    print("current time:-", ct)

In [4]:
class Red:
    #inicializar los atributos 
    def __init__(self, n, t, na):
        self.nombre = n
        self.topologia = t
        self.num_ataques = na

        # NÚMERO DE COMPONENTES CONECTADOS
        self.ncc_aleatorio = list()
        self.ncc_degree = list()
        self.ncc_closeness = list()
        self.ncc_harmonic = list()
        self.ncc_pagerank = list()
        self.ncc_betweenness = list()

        # PROPORCIÓN DEL COMPONENTE GIGANTE
        self.pcg_aleatorio = list()
        self.pcg_degree = list()
        self.pcg_closeness = list()
        self.pcg_harmonic = list()
        self.pcg_pagerank = list()
        self.pcg_betweenness = list()

        # DIÁMETRO
        self.diametro_aletorio = list()
        self.diametro_degree = list()
        self.diametro_closeness = list()
        self.diametro_harmonic = list()
        self.diametro_pagerank = list()
        self.diametro_betweenness = list()

        # LONGITUD PROMEDIO DE LA TRAYECTORIA
        self.aspl_aleatorio = list()
        self.aspl_degree = list()
        self.aspl_closeness = list()
        self.aspl_harmonic = list()
        self.aspl_pagerank = list()
        self.aspl_betweenness = list()

        # PROBABILIDAD DE CONEXIÓN
        self.pc_aleatorio = list()
        self.pc_degree = list()
        self.pc_closeness = list()
        self.pc_harmonic = list()
        self.pc_pagerank = list()
        self.pc_betweenness = list()

    
    ### M E T R I C A S ###

    def num_componentes_conectados(self, G):
        #A la variable num_cc se le asigna el valor de componentes conectados.
        print("Componentes conectados")
        imprimir_hora()
        num_cc = nx.number_connected_components(G)
        return num_cc
    
    def prop_componente_gigante(self, G):
        print("Proporcion del componente gigagnte")
        imprimir_hora()
        componentes_conectados = sorted(nx.connected_components(G), key=len, reverse=True) # Duda, por qué los ordena?
        G0 = G.subgraph(componentes_conectados[0]) #Componente Gigant    
        tcg = len(G0) #Tamaño del componente Gigante, cuenta los elementos de la lista G0
        n = len(nx.nodes(G)) # Cuenta el numero de nodos que tiene la red G
        pcg = tcg / n #Proporción del componente gigante
        return pcg

    def diametro(self, G):
        print("Diametro")
        imprimir_hora()
        componentes_conectados = sorted(nx.connected_components(G), key=len, reverse=True)
        G0 = G.subgraph(componentes_conectados[0])
        diameter = nx.diameter(G0)
        return diameter

    def trayectoria_promedio(self, G):
        print("Trayectoria promedio")
        imprimir_hora()
        componentes_conectados = sorted(nx.connected_components(G), key=len, reverse=True)
        G0 = G.subgraph(componentes_conectados[0])
        aspl = nx.average_shortest_path_length(G0) #longitud de ruta más corta promedio
        return aspl

    def probabilidad_de_conexion(self, G):
        print("Probabilidad de conexión")
        imprimir_hora()
        componentes = sorted(nx.connected_components(G))
        sumar_componentes = []
        sumar_probabilidad = []
        for x in componentes:
            y = (len(x))
            sumar_componentes.append(y)
            sumar_probabilidad.append((y * (y - 1))/2)
        resultado = sum(sumar_componentes)
        if resultado > 1:
            probabilidad = sum(sumar_probabilidad) / ((resultado * (resultado - 1))/ 2)
        else:
            probabilidad = 0
        return probabilidad

    
    ### A T A Q U E S ###    
    def centrality(self, G, tipo):
        H = G.copy()
        if tipo == 0:
            nodo_remover = random.sample(H.nodes, 1) #Elegir un nodo aleatorio
        elif tipo == 1:
            max_centrality = max(H.degree, key=lambda x: x[1])[0]
            nodo_remover = [max_centrality]
        elif tipo == 2:
            centralidad = nx.closeness_centrality(H) #Hacer la centralidad de cada nodo (diccionario)
        elif tipo == 3:
            centralidad = nx.harmonic_centrality(H) #Hacer la centralidad de cada nodo (diccionario) 
        elif tipo == 4:
            centralidad = nx.pagerank(H) #Por defecto el maximo de iteraciones es 100
        elif tipo == 5:
            centralidad = nx.betweenness_centrality(H) #Hacer la centralidad de cada nodo (diccionario)
        if tipo >= 2 and tipo <=5:
            max_centrality = (max(centralidad, key=centralidad.get)) #Encontrar la llave con el valor más alto
            nodo_remover = [max_centrality]
        
        H.remove_nodes_from(nodo_remover) #Remover el nodo
        return H


    ### FUNCIÓN TIPO DE ATAQUE ###
    def ataque(self, tipo):
        G = self.topologia.copy()
        num_cc_lista = [1] # Número de componentes conectados
        prop_cg_lista = [1] # Proporción del componente gigante
        ataques_lista = [0]
        ataque = 0
        diametro_lista = [self.diametro(G)] # Diámetro        
        longitud_promedio = [self.trayectoria_promedio(G)] # Longitud de la trayectoria Promedio
        probabilidad_conexion = [self.probabilidad_de_conexion(G)] # Probabilidad de conexión de nodos

        for x in range(self.num_ataques):
            print("*** numero ataque ", x, " ***")
            G = self.centrality(G, tipo)
            num_cc_lista.append(self.num_componentes_conectados(G))
            prop_cg_lista.append(self.prop_componente_gigante(G))
            ataque = ataque + 1 # ataque += 1
            ataques_lista.append(ataque/len(nx.nodes(self.topologia))) # Proporción de nodos eliminados   
            diametro_lista.append(self.diametro(G)) # Lista Diámetro    
            probabilidad_conexion.append(self.probabilidad_de_conexion(G)) #Lista probabilidad de conexión
            longitud_promedio.append(self.trayectoria_promedio(G)) # Lista Trayectoria Promedio


        if tipo == 0:
            self.ncc_aleatorio = num_cc_lista
            self.pcg_aleatorio = prop_cg_lista
            self.diametro_aletorio = diametro_lista
            self.aspl_aleatorio = longitud_promedio
            self.pc_aleatorio = probabilidad_conexion
        
        elif tipo == 1:
            self.ncc_degree = num_cc_lista
            self.pcg_degree = prop_cg_lista
            self.diametro_degree = diametro_lista
            self.aspl_degree = longitud_promedio
            self.pcg_degree = probabilidad_conexion
        
        elif tipo == 2:
            self.ncc_closeness = num_cc_lista
            self.pcg_closeness = prop_cg_lista
            self.diametro_closeness = diametro_lista
            self.aspl_closeness = longitud_promedio
            self.pc_closeness = probabilidad_conexion
        
        elif tipo == 3:
            self.ncc_harmonic = num_cc_lista
            self.pcg_harmonic = prop_cg_lista
            self.diametro_harmonic = diametro_lista
            self.aspl_harmonic = longitud_promedio
            self.pc_harmonic = probabilidad_conexion
        
        elif tipo == 4:
            self.ncc_pagerank = num_cc_lista
            self.pcg_pagerank = prop_cg_lista
            self.diametro_pagerank = diametro_lista
            self.aspl_pagerank = longitud_promedio
            self.pc_pagerank = probabilidad_conexion
        
        elif tipo == 5:
            self.ncc_betweenness = num_cc_lista
            self.pcg_betweenness = prop_cg_lista
            self.diametro_betweenness = diametro_lista
            self.aspl_betweenness = longitud_promedio
            self.pc_betweenness = probabilidad_conexion
        
        return (ataques_lista, num_cc_lista, prop_cg_lista, diametro_lista, longitud_promedio, probabilidad_conexion)        

In [5]:
def atacar_red(grafo, nombre_grafo, num_ataques):
    r1 = Red(nombre_grafo, grafo, num_ataques)
    # declarar un diccionario por cada metrica
    diametro = dict()
    num_cc = dict()
    proporcion_cg = dict()
    aspl = dict()
    prob_conexion = dict()

    tipos_ataques = ["aleatorio", "grado", "cercania", "harmonica", "pagerank", "intermediacion"]

    for i in range(6):
        print("*************************")
        print("Tipo de ataque " + str(i))
        imprimir_hora()
        (ataques_lista, num_cc_lista, prop_cg_lista, diametro_lista, longitud_promedio, probabilidad_conexion) = r1.ataque(i)
        #Almacenar las metricas en sus diccionarios
        diametro[tipos_ataques[i]] = diametro_lista
        num_cc[tipos_ataques[i]] = num_cc_lista
        proporcion_cg[tipos_ataques[i]] = prop_cg_lista
        aspl[tipos_ataques[i]] = longitud_promedio
        prob_conexion[tipos_ataques[i]] = probabilidad_conexion

    #Agregar la lista de ataques a cada diccionario
    diametro["num_ataques"] = ataques_lista
    num_cc["num_ataques"] = ataques_lista
    proporcion_cg["num_ataques"] = ataques_lista
    aspl["num_ataques"] = ataques_lista
    prob_conexion["num_ataques"] = ataques_lista

    #Almacenar cada diccionario (son 6) en un csv
    pd.DataFrame(diametro).to_csv('resultados\\diametro\\' + nombre_grafo + '.csv', index=False)
    pd.DataFrame(num_cc).to_csv('resultados\\numero_cc\\' + nombre_grafo + '.csv', index=False)
    pd.DataFrame(proporcion_cg).to_csv('resultados\\proporcion_cg\\' + nombre_grafo + '.csv', index=False)
    pd.DataFrame(aspl).to_csv('resultados\\aspl\\' + nombre_grafo + '.csv', index=False)
    pd.DataFrame(prob_conexion).to_csv('resultados\\prob_conexion\\' + nombre_grafo + '.csv', index=False)

In [18]:
for r in (1, 2):
    regla = "r" + str(r)
    for e in (1, 2, 4, 8, 16, 32):
        enlace = "enlaceD" + str(e)
        for x in (1, 2, 3, 4, 5, 6, 7, 8, 9, 10):
            experimento = "experimento" + str(x)
            nombre_grafo = regla + "_" + enlace + "_" + experimento + "_ciclo30.graphml"
            ruta_grafo = "graphs//" + regla + "//" + enlace + "//" + experimento + "//" + nombre_grafo

            grafo = nx.read_graphml(ruta_grafo)
            print("Nombre grafo: " + nombre_grafo)
            print("Ruta grafo: " + ruta_grafo)

            atacar_red(grafo, regla + "_" + enlace + "_" + experimento + "_ciclo30", 2499)

Nombre grafo: r1_enlaceD1_experimento1_ciclo30.graphml
Ruta grafo: graphs//r1//enlaceD1//experimento1//r1_enlaceD1_experimento1_ciclo30.graphml
*************************
Tipo de ataque 0
current time:- 2023-02-26 17:14:22.272584
Diametro
current time:- 2023-02-26 17:14:22.335414


KeyboardInterrupt: 